In [1]:
include("../src/JuliaMBD.jl")

Main.JuliaMBD

In [2]:
using DifferentialEquations
using Plots
import .JuliaMBD
import LookupTable

In [11]:
@macroexpand JuliaMBD.@parameter m x

:(addparameter!(m, :x, x, Auto))

In [14]:
@macroexpand JuliaMBD.@parameter m begin
    x::Float64 = 10
    y = 10
end

quote
    #= /workspaces/JuliaMBD/examples/macrotest.ipynb:2 =#
    addparameter!(m, :x, 10, Float64)
    #= /workspaces/JuliaMBD/examples/macrotest.ipynb:3 =#
    addparameter!(m, :y, 10, Auto)
end

In [17]:
@macroexpand JuliaMBD.@block m x = x

quote
    #= /workspaces/JuliaMBD/src/_macro.jl:39 =#
    x = x
    #= /workspaces/JuliaMBD/src/_macro.jl:40 =#
    add!(m, x)
end

In [66]:
@macroexpand JuliaMBD.@scope m begin
    y.u => x
end

quote
    #= /workspaces/JuliaMBD/examples/macrotest.ipynb:2 =#
    JuliaMBD.addscope!(m, :x, getport(y, :u))
end

In [33]:
Meta.dump(:(a.x))

Expr
  head: Symbol .
  args: Array{Any}((2,))
    1: Symbol a
    2: QuoteNode
      value: Symbol x


In [34]:
uu = Expr(:., :a, Expr(:quote, :x))

:(a.x)

In [48]:
length(uu.args)

2

In [60]:
typeof(uu.args[2])

QuoteNode

In [64]:
JuliaMBD._togetport(:(x.a))

:(getport(x, :a))

In [42]:
uu = :(x.a)

:(x.a)

In [69]:
@macroexpand @JuliaMBD.connect m x.out => a.in

:(JuliaMBD.LineSignal(getport(x, :out), getport(a, :in)))

In [70]:
@macroexpand @JuliaMBD.connect m begin
    x.out => a.in
    @JuliaMBD.parameter m begin
        v
    end
end

quote
    #= /workspaces/JuliaMBD/examples/macrotest.ipynb:2 =#
    JuliaMBD.LineSignal(getport(x, :out), getport(a, :in))
    #= /workspaces/JuliaMBD/examples/macrotest.ipynb:3 =#
    begin
        #= /workspaces/JuliaMBD/examples/macrotest.ipynb:4 =#
        JuliaMBD.addparameter!(m, :v, x, Auto)
    end
end

In [3]:
import JuliaMBD: @model, @parameter, @connect

In [4]:
@macroexpand @model :FF begin
@parameter begin
        x::Float32
        y
    end
    @connect begin
        x.u => y
    end
end

:(function (:FF)(; x::Float32 = 0, y = :y)
      #= /workspaces/JuliaMBD/examples/macrotest.ipynb:2 =#
      begin
          #= /workspaces/JuliaMBD/examples/macrotest.ipynb:3 =#
          JuliaMBD.addparameter!(tmp, :x, x, Float32)
          #= /workspaces/JuliaMBD/examples/macrotest.ipynb:4 =#
          JuliaMBD.addparameter!(tmp, :y, x, Auto)
      end
      #= /workspaces/JuliaMBD/examples/macrotest.ipynb:6 =#
      begin
          #= /workspaces/JuliaMBD/examples/macrotest.ipynb:7 =#
          JuliaMBD.LineSignal(getport(x, :u), y)
      end
  end)

In [100]:
xs = quote
    @model :FF begin
@parameter begin
        x = 10
        y
    end
    @connect begin
        x.u => y
    end
end
end
res = []
JuliaMBD.getargs(xs, res)


In [101]:
res

2-element Vector{Any}:
 :($(Expr(:kw, :x, 10)))
 :y

In [82]:
xs.args[2].args

4-element Vector{Any}:
 Symbol("@model")
 :(#= /workspaces/JuliaMBD/examples/macrotest.ipynb:2 =#)
 :(:FF)
 quote
    #= /workspaces/JuliaMBD/examples/macrotest.ipynb:3 =#
    #= /workspaces/JuliaMBD/examples/macrotest.ipynb:3 =# @parameter begin
            #= /workspaces/JuliaMBD/examples/macrotest.ipynb:4 =#
            x
            #= /workspaces/JuliaMBD/examples/macrotest.ipynb:5 =#
            y
        end
    #= /workspaces/JuliaMBD/examples/macrotest.ipynb:7 =#
    #= /workspaces/JuliaMBD/examples/macrotest.ipynb:7 =# @connect begin
            #= /workspaces/JuliaMBD/examples/macrotest.ipynb:8 =#
            x.u => y
        end
end

In [7]:
zero(Vector{Float64})

MethodError: MethodError: no method matching zero(::Type{Vector{Float64}})
Closest candidates are:
  zero(!Matched::Union{Type{P}, P}) where P<:Dates.Period at /opt/julia/share/julia/stdlib/v1.8/Dates/src/periods.jl:53
  zero(!Matched::LinearAlgebra.Diagonal{T, StaticArraysCore.SVector{N, T}}) where {N, T} at ~/.julia/packages/StaticArrays/jA1zK/src/SDiagonal.jl:41
  zero(!Matched::LinearAlgebra.Diagonal) at /opt/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/special.jl:381
  ...